# RO Flowsheet Build

This tutorial will demostrate how to create a full flowsheet with the WaterTAP reverse osmosis model. It will combine unit models and features demonstrated in all previous tutorials to this point in the Academy.

<center><img src="graphics/RO_flowsheet.png" width="600" /></center>


To build this flowsheet, this tutorial will follow the general workflow presented below. Though we will be using specific unit models, the process is generalizable to any treatment train. 

<center><img src="graphics/watertap-flowsheet-workflow.png" width="600" /></center>

## Step 1: Import Modules

<center><img src="graphics/workflow-step1.png" width="600" /></center>

In [ ]:
# Imports from Pyomo
from pyomo.environ import (
    ConcreteModel,
    Var,
    Constraint,
    Objective,
    value,
    assert_optimal_termination,
    TransformationFactory,
    units as pyunits,
)
from pyomo.network import Arc

# Imports from IDAES
from idaes.core import FlowsheetBlock
from idaes.models.unit_models import Feed, Product
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.scaling import calculate_scaling_factors, set_scaling_factor
from idaes.core.util.initialization import propagate_state

# Imports from WaterTAP
from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.unit_models.pressure_changer import Pump
from watertap.unit_models.reverse_osmosis_0D import (
    ReverseOsmosis0D,
    ConcentrationPolarizationType,
    MassTransferCoefficient,
    PressureChangeType,
)
from watertap.core.solvers import get_solver

## Step 2: Build Model

### Step 2a: Create Model Object
### Step 2b: Add Flowsheet to Model
### Step 2c: Add Property Packages to Flowsheet

<center><img src="graphics/workflow-step2abc.png" width="600" /></center>

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

### Step 2d: Add Unit Models to Flowsheet

<center><img src="graphics/workflow-step2d.png" width="600" /></center>

Unit models can be added to the flowsheet in any order. All unit models will have at least one configuration argument `property_package` that must be specified. Most WaterTAP process models are designed to work with a specific unit model, but others (like `Feed` and `Product`) can work with any property package.

In [ ]:
# Add feed
m.fs.feed = Feed(property_package=m.fs.properties)

# Add pump
m.fs.pump = Pump(
    property_package=m.fs.properties,
)

# Add reverse osmosis unit
m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    has_pressure_change=True,
    pressure_change_type=PressureChangeType.calculated,
    mass_transfer_coefficient=MassTransferCoefficient.calculated,
    concentration_polarization_type=ConcentrationPolarizationType.calculated,
)

# Add product and brine streams
m.fs.product = Product(property_package=m.fs.properties)
m.fs.brine = Product(property_package=m.fs.properties)

### Step 2e: Define Unit Model Connectivity

<center><img src="graphics/workflow-step2e.png" width="600" /></center>

All unit models will contain `Port` that enable their state blocks to "connect" to the state blocks of other unit models. We establish these connections by creating an `Arc` between a `Port` on each unit model. 

In [ ]:
m.fs.feed_to_pump = Arc(source=m.fs.feed.outlet, destination=m.fs.pump.inlet)
m.fs.pump_to_RO = Arc(source=m.fs.pump.outlet, destination=m.fs.RO.inlet)
m.fs.RO_to_product = Arc(source=m.fs.RO.permeate, destination=m.fs.product.inlet)
m.fs.RO_to_brine = Arc(source=m.fs.RO.retentate, destination=m.fs.brine.inlet)

### Step 2f: Expand Arcs

<center><img src="graphics/workflow-step2f.png" width="600" /></center>

After establishing these connection, the arcs are expanded to create the constraints between all the state variables.

In [ ]:
TransformationFactory("network.expand_arcs").apply_to(m)

### Step 2g: Add Variables, Constraints, and Objectives

<center><img src="graphics/workflow-step2g.png" width="600" /></center>

The last step before model specification is to add any additional variables, constraints, or objectives that are desired on the model. This is an optional step, but allows the user to create custom relationships on the fly.

For demonstration purposes, below we create a variable and constraint that defines the RO flux in LMH. 

In [ ]:
m.fs.RO.flux_LMH = Var(
    initialize=20.0,
    bounds=(0, 45),
    units=pyunits.liter / (pyunits.m**2 * pyunits.hr),
    doc="Water flux through the membrane in LMH",
)

m.fs.RO.flux_LMH_constraint = Constraint(
    expr=m.fs.RO.flux_LMH
    == pyunits.convert(
        m.fs.RO.mixed_permeate[0].flow_vol_phase["Liq"] / m.fs.RO.area,
        to_units=pyunits.liter / (pyunits.m**2 * pyunits.hr),
    )
)

## Step 3: Specify Model

<center><img src="graphics/workflow-step3.png" width="600" /></center>

As before, we must provide parameter values for all the degrees of freedom on the model. These can be fixed in any order, but we start with the feed and work to the end of the train in this example.

The `brine` and `product` blocks are fully specfied by their connected arcs.

In [ ]:
# Feed, 4 degrees of freedom
m.fs.feed.properties[0].flow_vol_phase["Liq"].fix(1e-3)
m.fs.feed.properties[0].conc_mass_phase_comp["Liq", "TDS"].fix(35)
m.fs.feed.properties[0].pressure.fix(101325)
m.fs.feed.properties[0].temperature.fix(273.15 + 25)

# Pump, 2 degrees of freedom
m.fs.pump.efficiency_pump.fix(0.80)
m.fs.pump.control_volume.properties_out[0].pressure.fix(75 * pyunits.bar)

# RO unit, 7 degrees of freedom
m.fs.RO.A_comp.fix(4.2e-12)
m.fs.RO.B_comp.fix(3.5e-8)
m.fs.RO.recovery_vol_phase[0, "Liq"].fix(0.5)
m.fs.RO.feed_side.velocity[0, 0].fix(0.15)
m.fs.RO.feed_side.channel_height.fix(1e-3)
m.fs.RO.feed_side.spacer_porosity.fix(0.97)
m.fs.RO.permeate.pressure[0].fix(101325)

print("DOF = ", degrees_of_freedom(m))

# Touch any desired properties for reporting
m.fs.brine.properties[0].flow_vol_phase
m.fs.brine.properties[0].conc_mass_phase_comp
m.fs.product.properties[0].flow_vol_phase
m.fs.product.properties[0].conc_mass_phase_comp

## Step 4: Scale Model

<center><img src="graphics/workflow-step4.png" width="600" /></center>


Scaling ensures that all numerical quantities fall within similar, moderate magnitudes, improving model stability.

First we set default scaling factors for our state variables. Then we set a custom scaling factor for the membrane area. Lastly, we use `calculate_scaling_factors` which will set scaling factors for the remaining variables on the model.

In [ ]:
# Set default property values
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "TDS"))

# Set unit model values
set_scaling_factor(m.fs.pump.control_volume.work, 1e-3)
set_scaling_factor(m.fs.RO.area, 1e-2)

# Calculate and propagate scaling factors
calculate_scaling_factors(m)

## Step 5: Initialize Model

<center><img src="graphics/workflow-step5.png" width="600" /></center>

After the model is scaled, we move to initialization. Initialization can be thought of solving each model in a vacuum, without any relation to the up or downstream effects. For the other steps in this workflow, the order with respect to the process flow did not matter. However, initialization must proceed from the beginning to the end of the treatment train.

Starting with the `feed` block, the model is set to an initial point. After solving the feed block, we _propagate the state_ from the feed `outlet` to the pump `inlet`. When we know the state of the pump inlet, we can solve the pump model and further propagate down stream, and so on.

In [ ]:
# Get WaterTAP solver
solver = get_solver()

# solve feed
solver.solve(m.fs.feed)

# Propagate state from feed to pump
propagate_state(m.fs.feed_to_pump)
# Initialize pump
m.fs.pump.initialize()

# Propagate state from pump to RO
propagate_state(m.fs.pump_to_RO)
# Initialize RO
m.fs.RO.initialize()

# Propagate state from RO to brine and product
propagate_state(m.fs.RO_to_brine)
propagate_state(m.fs.RO_to_product)

# Initialize brine and product
m.fs.brine.initialize()
m.fs.product.initialize()

## Step 6: Solve Model

<center><img src="graphics/workflow-step6.png" width="600" /></center>

In [ ]:
# Solve model
results = solver.solve(m)
assert_optimal_termination(results)
print(
    f"Solver Status: {results.solver.status}, Termination Condition: {results.solver.termination_condition}"
)

# Try It Yourself

Using the existing model, unfix the RO recovery and pressure and resolve the model. Then check the resulting recovery and pressure.

In [ ]:
print(
    f"Old Recovery: {value(m.fs.RO.recovery_vol_phase[0, 'Liq'])*100:.2f}%, Old Pump Outlet Pressure: {value(m.fs.pump.control_volume.properties_out[0].pressure/1e5):.2f} bar"
)

# Unfix recovery and pump outlet pressure and re-solve


# results = solver.solve(m)
# assert_optimal_termination(results)
# print(
#     f"Solver Status: {results.solver.status}, Termination Condition: {results.solver.termination_condition}"
# )
# print(
#     f"New Recovery: {value(m.fs.RO.recovery_vol_phase[0, 'Liq'])*100:.2f}%, New Pump Outlet Pressure: {value(m.fs.pump.control_volume.properties_out[0].pressure/1e5):.2f} bar"
# )